In [39]:
import numpy
from scipy.stats import laplace, norm, t
import scipy
import math
import numpy as np
from scipy.special import logsumexp

VARIANCE = 2.0

normal_scale = math.sqrt(VARIANCE)
student_t_df = (2 * VARIANCE) / (VARIANCE - 1)
laplace_scale = VARIANCE / 2

HYPOTHESIS_SPACE = [norm(loc=0.0, scale=math.sqrt(VARIANCE)),
                    laplace(loc=0.0, scale=laplace_scale),
                    t(df=student_t_df)]

PRIOR_PROBS = np.array([0.35, 0.25, 0.4])


def generate_sample(n_samples, seed=None):
    """ data generating process of the Bayesian model """
    random_state = np.random.RandomState(seed)
    hypothesis_idx = np.random.choice(3, p=PRIOR_PROBS)
    dist = HYPOTHESIS_SPACE[hypothesis_idx]
    return dist.rvs(n_samples, random_state=random_state)


""" Solution """




def log_posterior_probs(x):
    """
    Computes the log posterior probabilities for the three hypotheses, given the data x

    Args:
        x (np.ndarray): one-dimensional numpy array containing the training data
    Returns:
        log_posterior_probs (np.ndarray): a numpy array of size 3, containing the Bayesian log-posterior probabilities
                                          corresponding to the three hypotheses
    """

    list1 = []
    list2 = []
    
    for (dist,prior) in zip(HYPOTHESIS_SPACE,PRIOR_PROBS):
        temp = dist.pdf(x)
        # print(temp)
        combined = np.sum(np.log(temp))
        # print(combined)
        withprior = combined + np.log(prior)
        # print("withprior: ", withprior)
        list1.append(withprior)


    m = np.max(list1)
    for logprob in list1:
        # print("m: ", m)
        # print("logprob: ", logprob)
        # print("logprob-m: ", logprob-m)
        shifted = logprob-m
        # print("shifted: ", shifted)
        converted = np.exp(shifted)
        # print("converted: ", converted)
        list2.append(converted)
    
    sum = np.array(list2).sum()

    p = [list2[i]/sum for i in range(0,3)]
    log_p = np.log(p)
    return log_p


def posterior_probs(x):
    return np.exp(log_posterior_probs(x))


In [40]:
""" sample from Laplace dist """
dist = HYPOTHESIS_SPACE[1]
x = dist.rvs(1000, random_state=28)

print("Posterior probs for 1 sample from Laplacian")
p = posterior_probs(x[:1])
print("Normal: %.4f , Laplace: %.4f, Student-t: %.4f\n" % tuple(p))

print("Posterior probs for 50 samples from Laplacian")
p = posterior_probs(x[:50])
print("Normal: %.4f , Laplace: %.4f, Student-t: %.4f\n" % tuple(p))

print("Posterior probs for 1000 samples from Laplacian")
p = posterior_probs(x[:1000])
print("Normal: %.4f , Laplace: %.4f, Student-t: %.4f\n" % tuple(p))

print("Posterior for 100 samples from the Bayesian data generating process")
x = generate_sample(n_samples=100)
p = posterior_probs(x)
print("Normal: %.4f , Laplace: %.4f, Student-t: %.4f\n" % tuple(p))

Posterior probs for 1 sample from Laplacian
Normal: 0.3239 , Laplace: 0.2441, Student-t: 0.4320

Posterior probs for 50 samples from Laplacian
Normal: 0.2550 , Laplace: 0.3388, Student-t: 0.4062

Posterior probs for 1000 samples from Laplacian
Normal: 0.0000 , Laplace: 1.0000, Student-t: 0.0000

Posterior for 100 samples from the Bayesian data generating process
Normal: 0.9735 , Laplace: 0.0023, Student-t: 0.0242

